In [ ]:
# %%

import pandas as pd

from source.utils import FREQ_DIR, RESULT_DIR, UCS_DIR, confirm_dir
from source.utils.associate import (BINARY_ASSOC_ARGS, add_extra_am,
                                    associate_ucs, confirm_basic_ucs)
from source.utils.associate import convert_ucs_to_csv as ucs2csv
from source.utils.associate import get_associations_csv as init_am, AM_DF_DIR
from source.utils.associate import manipulate_ucs, seek_readable_ucs, adjust_assoc_columns
pd.set_option('display.float_format', '{:,.2f}'.format)

/home/arh234/anaconda3/envs/dev-sanpi/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# %% [markdown]

 set parameters:
 - `UNIT` options:
   - `'Bigr'`
   - `'Adv'`
   - `'Adj'`
 - `PAT_DIR` options:
   - any directory in `results/freq_out/` containing a `ucs_format/` subdir with a `Trig*.tsv` frequency table
   - developed with the following in mind:
     - `POSmirror`
     - `NEGmirror`
     - `ANYmirror` (which has been populated by simple concatenation in the shell)

In [ ]:
UNIT = 'Adj'
# UNIT = 'Adv'
# UNIT = 'Bigr' 
# PAT_DIR = 'POSmirror'
PAT_DIR = 'NEGmirror'
# PAT_DIR = 'ANYmirror'
# FRQ_FLOOR = 3
# FRQ_FLOOR = 10
FRQ_FLOOR = 20
# FRQ_FLOOR = 50
# FRQ_FLOOR = 100
TRIG_TSV = FREQ_DIR.joinpath(
    f'{PAT_DIR}/ucs_format/Trig{UNIT}_frq-thrMIN-7.35f.tsv')
FOCUS = ['f', 'unexpected_f',
         'conservative_log_ratio',
         'am_p1_given2', 'am_p2_given1',
         'am_p1_given2_simple', 'am_p2_given1_simple',
         'am_log_likelihood',
        #  'mutual_information', 'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11',
         'l1', 'l2']

In [ ]:
# %%

TRIGGER_POLARITY = {
    'positive': {'all',
                 'always',
                 'both',
                 'either',
                 'every',
                 'everybody',
                 'everyone',
                 'everything',
                 'many',
                 'often',
                 'or',
                 'some',
                 'somebody',
                 'someone',
                 'something',
                 'somethings',
                 'sometimes'},
    'negative': {'barely',
                 'hardly',
                 'neither',
                 'never',
                 'no',
                 'nobody',
                 'none',
                 'nor',
                 'nothing',
                 'seldom',
                 'rarely',
                 'scarcely'},

}

In [ ]:
# %%

TRIGGER_QUANT = {
    'existential': {
        'some',
        'somebody',
        'someone',
        'something',
        'somethings',
        'either',
     'or',
        'sometimes'
    },
    'universal': {
        'all', 'every',
        'always',
        'everybody',
        'everyone',
        'both',
        'everything',
    },
    'not_exist': {
        'neither',
        'never',
        'no',
        'nobody',
        'none',
        'nor',
        'nothing',
    },
    'hedged_not_exist': {
        'barely',
        'hardly',
        'rarely',
        'scarcely',
        'seldom',
        'few'
    },
    'hedged_universal':{
     'many',
     'often',
     }
}

In [ ]:
# %%

def invert_set_dict(d: dict):
    return {v: k for k in d for v in d[k]}

# %% [markdown]

## 1. Run `seek_readable_ucs()` to generate consistent output path

In [ ]:
readable = seek_readable_ucs(min_freq=FRQ_FLOOR,
                             ucs_subdir='trigger_eval',
                             contained_counts_path=TRIG_TSV)
print(readable.relative_to(RESULT_DIR))

    > seeking `trigger_eval/NEGmirror/readable/TrigAdj_frq-thrMIN-7.35f_min20x*` frequency data and initial associations...
ucs/trigger_eval/NEGmirror/readable/TrigAdj_frq-thrMIN-7.35f_min20x.rsort-view_am-only.txt


# %% [markdown]

 Snippet of starting frequency data (`TRIG_TSV`)

In [ ]:
! head -5 {TRIG_TSV} | column -t

10804  never    late
8025   nothing  important
4901   never    early
4856   nothing  wrong
4677   never    important


# %% [markdown]

## 2. Run `confirm_basic_ucs()` (if needed)

In [ ]:
if not readable.is_file():
    basic_ucs_path = readable.parent.parent.joinpath(
        readable.name.replace('.rsort-view_am-only.txt', '.ds.gz'))
    print(f'Creating initial UCS table: `{basic_ucs_path.relative_to(RESULT_DIR)}')

    basic_ucs_path = confirm_basic_ucs(
        basic_ucs_path,
        freq_floor=FRQ_FLOOR,
        contained_counts_path=TRIG_TSV)

Creating initial UCS table: `ucs/trigger_eval/NEGmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz

## Creating initial UCS table...

```
( cat /share/compling/projects/sanpi/results/freq_out/NEGmirror/ucs_format/TrigAdj_frq-thrMIN-7.35f.tsv \ 
  | ucs-make-tables --types --threshold=20 /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz ) \ 
  && ucs-sort /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz BY f2- f1- INTO /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz
== Note ==
    N = total number of tokens/all counts summed
    V = total number of rows/number of unique l1+l2 combinations before filtering to 20+ tokens
+ time to make table → 00:00:00.200
```

Saving initial frequency table in readable .txt format...

```
ucs-print -o /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/readable/TrigAdj_frq-thrMIN-7.3

# %% [markdown]

 Excerpt of initial UCS table

In [ ]:
init_readable = UCS_DIR.joinpath(
    f'trigger_eval/{PAT_DIR}/readable'
).joinpath(f'{TRIG_TSV.name.replace(".tsv","")}_min{FRQ_FLOOR}x.init.txt')
! head -7 {init_readable}

      l1  l2                   f     f2      f1       N  
--------  ---------------  -----  -----  ------  ------  
   never  important         4677  14658  109723  289776  
 nothing  important         8025  14658   98788  289776  
    none  important         1259  14658   31021  289776  
     nor  important          140  14658   15546  289776  
      no  important          258  14658    8890  289776  


# %% [markdown]

## 3. Run `associate_ucs()` (if needed)

In [ ]:
if not readable.is_file():
    associate_ucs(basic_ucs_path)

transform_ucs_log = f'/share/compling/projects/sanpi/logs/associate/ucs//ucs-{PAT_DIR}_Trig{UNIT}_frq-thrMIN-7-35f_min{FRQ_FLOOR}x*.log'
! head -15 `ls -t1 {transform_ucs_log} | head -1`
! echo '...'
! tail -2 `ls -t1 {transform_ucs_log} | head -1`


Calculating UCS associations...

```
bash /share/compling/projects/sanpi/script/transform_ucs.sh /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz
> log will be saved to: /share/compling/projects/sanpi/logs/associate/ucs//ucs-NEGmirror_TrigAdj_frq-thrMIN-7-35f_min20x.2024-05-25_2222.log
...
```

+ time elapsed → 00:00:12.835
# Manipulating TrigAdj_frq-thrMIN-7-35f_min20x ucs table
path to this script: /share/compling/projects/sanpi/script/transform_ucs.sh
Sat May 25 22:22:49 EDT 2024
(TMP: /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/tmp/tmp_NEGmirror-20240525-222249.TrigAdj_frq-thrMIN-7-35f_min20x)
## Initial Contingency Info

DATA SET FILE:  /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdj_frq-thrMIN-7.35f_min20x.ds.gz

# Frequency signatures computed by the ucs-make-tables tool for relational cooccurrences.
# Sample size:  N = 289776 tokens,  V = 21562 pair types.
# A frequency thres

# %% [markdown]

### Define dictionary containing relevant vocab sizes
 !!! Warning This is a `#HACK`: \
     Rather than developing a command/code to retrieve the vocab sizes programmatically,
     I simply copied the values given in the log output of `transform_usc.sh`
     for each `PAT_DIR`+`UNIT` combination

 |      |   ANYmirror |   NEGmirror |   POSmirror |
 |:-----|------------:|------------:|------------:|
 | Bigr |     395,338 |      64,644 |     330,694 |
 | Adv  |      23,125 |       5,004 |      18,121 |
 | Adj  |      83,422 |      21,562 |      61,860 |

In [ ]:
VOCABS = {'ANYmirror': {'Adv': 23125, 'Bigr': 395338, 'Adj': 83422},
          'NEGmirror': {'Adv': 5004, 'Bigr': 64644, 'Adj': 21562},
          'POSmirror': {'Adv': 18121, 'Bigr': 330694, 'Adj': 61860}
          }  # HACK
VOCAB = VOCABS[PAT_DIR][UNIT]
pd.DataFrame(VOCABS)

,ANYmirror,NEGmirror,POSmirror
Adv,23125,5004,18121
Bigr,395338,64644,330694
Adj,83422,21562,61860


# %% [markdown]

## 4. Run `ucs_to_csv()` to convert `ucs/[PAT_DIR]/readable/*.txt` to format that `pandas` can parse as a dataframe

In [ ]:
! head -5 {readable}
csv_path = ucs2csv(readable)
print(f'CSV: `{csv_path.relative_to(RESULT_DIR)}`')

      l1  l2                   f           E11  am.log.likelihood  am.odds.ratio.disc   am.p1.given2   am.p2.given1  am.p1.given2.simple  am.p2.given1.simple      f1     f2       N  
--------  ---------------  -----  ------------  -----------------  ------------------  -------------  -------------  -------------------  -------------------  ------  -----  ------  
      no  dunk                33     1.0430816         221.113922         2.850171707   0.9400196605   0.0037084758         0.9705882353         0.0037120360    8890     34  289776  
    none  blind              190    23.8725188         670.600302         1.678689718   0.7455400213   0.0059973494         0.8520179372         0.0061248831   31021    223  289776  
    none  pleased           1298   167.8569930        4468.110921         1.620610923   0.7246757444   0.0407991646         0.8278061224         0.0418426227   31021   1568  289776  
UCS table text converted & saved as /share/compling/projects/sanpi/results/ucs/trigge

# %% [markdown]

##

In [ ]:
trig_amdf = pd.read_csv(csv_path).convert_dtypes()
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
0,no,dunk,33,1.04,221.11,2.85,0.94,0.00,0.97,0.00,8890,34,289776
1,none,blind,190,23.87,670.60,1.68,0.75,0.01,0.85,0.01,31021,223,289776
2,none,pleased,1298,167.86,"4,468.11",1.62,0.72,0.04,0.83,0.04,31021,1568,289776
3,no,pink,45,1.84,247.25,1.97,0.72,0.01,0.75,0.01,8890,60,289776
4,none,deaf,24,3.21,78.60,1.50,0.69,0.00,0.80,0.00,31021,30,289776
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1789,never,memorable,23,237.79,-423.87,-1.20,-0.34,-0.00,0.04,0.00,109723,628,289776
1790,never,wrong,220,"2,015.92","-3,500.17",-1.16,-0.34,-0.03,0.04,0.00,109723,5324,289776
1791,nothing,late,22,"3,718.66","-9,023.78",-2.42,-0.35,-0.06,0.00,0.00,98788,10908,289776
1792,never,surprising,22,343.43,-679.41,-1.38,-0.36,-0.00,0.02,0.00,109723,907,289776


In [ ]:
# %%

trig_amdf['key'] = (trig_amdf.l1 + '~' +
                    trig_amdf.l2).astype('string')
trig_amdf = trig_amdf.set_index('key')
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
key,,,,,,,,,,,,,
no~dunk,no,dunk,33,1.04,221.11,2.85,0.94,0.00,0.97,0.00,8890,34,289776
none~blind,none,blind,190,23.87,670.60,1.68,0.75,0.01,0.85,0.01,31021,223,289776
none~pleased,none,pleased,1298,167.86,"4,468.11",1.62,0.72,0.04,0.83,0.04,31021,1568,289776
no~pink,no,pink,45,1.84,247.25,1.97,0.72,0.01,0.75,0.01,8890,60,289776
none~deaf,none,deaf,24,3.21,78.60,1.50,0.69,0.00,0.80,0.00,31021,30,289776
...,...,...,...,...,...,...,...,...,...,...,...,...,...
never~memorable,never,memorable,23,237.79,-423.87,-1.20,-0.34,-0.00,0.04,0.00,109723,628,289776
never~wrong,never,wrong,220,"2,015.92","-3,500.17",-1.16,-0.34,-0.03,0.04,0.00,109723,5324,289776
nothing~late,nothing,late,22,"3,718.66","-9,023.78",-2.42,-0.35,-0.06,0.00,0.00,98788,10908,289776


# %% [markdown]

## 6. Save to `./results/assoc_df/`

In [ ]:
df_csv_path = AM_DF_DIR.joinpath(
    str(csv_path.relative_to(UCS_DIR))
    .replace('/readable', '')
    .replace('.rsort-view_am-only', ''))

if not df_csv_path.is_file():
    confirm_dir(df_csv_path.parent)
    trig_amdf.to_csv(df_csv_path)

df_pkl_path = df_csv_path.with_suffix('.pkl.gz')
if not df_pkl_path.is_file():
    trig_amdf.to_pickle(df_csv_path.with_suffix('.pkl.gz'))

# %% [markdown]

## 7. Add additional AM via `add_extra_am()`

In [ ]:
ex_trig_amdf = add_extra_am(df=trig_amdf,
                            verbose=True,
                            vocab=VOCAB,
                            metrics=['t_score', 'mutual_information']
                            ).convert_dtypes()


Preview of Extended Measures (rounded)

| key                  |   t_score |   mutual_information |   deltaP_min |   deltaP_max |   deltaP_max_abs |   deltaP_product |   unexpected_f |   unexpected_ratio |
|:---------------------|----------:|---------------------:|-------------:|-------------:|-----------------:|-----------------:|---------------:|-------------------:|
| no~dunk              |      5.56 |                 1.50 |         0.00 |         0.94 |             0.94 |             0.00 |          31.96 |               0.97 |
| none~blind           |     12.05 |                 0.90 |         0.01 |         0.75 |             0.75 |             0.00 |         166.13 |               0.87 |
| none~pleased         |     31.37 |                 0.89 |         0.04 |         0.72 |             0.72 |             0.03 |       1,130.14 |               0.87 |
| no~pink              |      6.43 |                 1.39 |         0.01 |         0.72 |             0.72 |             0.00 |  

#%% [markdown]

 Add trigger features as columns: polarity and quantification type

In [ ]:
def add_feature(triggers:pd.Series, 
                group_dict: dict) -> pd.Series:
    assign_dict = invert_set_dict(group_dict)
    return triggers.apply(lambda x: assign_dict[x] if x in assign_dict.keys() else '')

ex_trig_amdf['polarity'] = add_feature(ex_trig_amdf.l1, TRIGGER_POLARITY)
ex_trig_amdf['quant'] = add_feature(ex_trig_amdf.l1, TRIGGER_QUANT)

# %% [markdown]

 Save extended AM tables to `extra/` subdirectory if not already saved

In [ ]:
df_extra_csv = df_csv_path.parent / 'extra' / \
    df_csv_path.name.replace('.csv', '_extra.csv')
print(df_extra_csv)
if not df_extra_csv.is_file():
    confirm_dir(df_extra_csv.parent)
    ex_trig_amdf.to_csv(df_extra_csv)

df_extra_pkl = df_extra_csv.with_suffix('.pkl.gz')
if not df_extra_pkl.is_file():
    ex_trig_amdf.to_pickle(df_extra_pkl)

/share/compling/projects/sanpi/results/assoc_df/trigger_eval/NEGmirror/extra/TrigAdj_frq-thrMIN-7.35f_min20x_extra.csv


In [ ]:
# %%

ex_trig_full = ex_trig_amdf.copy()
ex_trig_abbr = adjust_assoc_columns(
    ex_trig_amdf[[c for c in ['polarity', 'quant'] + FOCUS if c in ex_trig_amdf.columns]]).sort_values('LRC', ascending=False)
cols = ex_trig_abbr.columns

# %% [markdown]

 Define lexical items with given lean shown in binary environment evaluation

In [ ]:
pos_prone = {
    'Adj': [
        'unrelated',
        'unable',
        'akin',
        'larger',
        'different',
        'familiar',
        'similar',
        'likely', 
        'brief', 
        'unaware'
    ],
    'Adv': [
        'slightly', 
        'definitely',
        'utterly',
        # LRC top
        'pretty', 
        'rather', 
        'plain', 
        'fairly', 
        'somewhat', 
        'otherwise', 
        'downright',
        'relatively', 
        # G2 top
        # 'very', 
        # 'even', 
        # 'just', 
        # dP1 top (and odds ratio disc)
        'plain', 
        'maybe'
    ], 
    'Bigr': [
        # G2 top
        'completely_different', 
        'too_familiar',
        'even_better',
        # dP1 top
        'quite_different', 
        'too_real', 
        'well_aware', 
        # LRC top
        'too_common', 
        'entirely_different'
        ]}
neg_prone = {
    'Bigr': [
        # LRC top
        'quite_sure',
        'really_sure', 
        'too_early', 
        'too_pleased', 
        'too_fancy',
        # dP1 top
        'entirely_sure', 
        'ever_easy', 
        'ever_perfect', 
        'particularly_surprising',
        'particularly_new',
        # G2 top
        'too_late', 
        'more_important',
        'so_easy', 
        'as_good', 
        'too_old'
        ], 
    'Adv': [
        'yet',
        #LRC top
        'ever',
        'any', 
        'longer', 
        'necessarily',
        'that',
        #dP1 top
        'before', 
        'wise',  #? How is this used as an adverb?
        'earthly',
        'remotely',
        'exactly',
        # G2 top
        'particularly', 
        'too',
        # 'inherently'
        ],
    'Adj': [
        # LRC top
        'early', 
        'late', 
        'fancy', 
        'alone', 
        'sure',
        # dP1 top
        'shabby',
        'demoralizing',
        'alone',
        'aggravating',
        'groundbreaking',
        'eventful',
        #G2 top
        'important', 
        'frustrating',
        'evident', 
        'certain'
        ]
    }
def sort_prone_by_f2(prone_list, amdf):
    return amdf.copy().loc[amdf.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
pos_prone[UNIT] = sort_prone_by_f2(pos_prone[UNIT], ex_trig_abbr)
neg_prone[UNIT] = sort_prone_by_f2(neg_prone[UNIT], ex_trig_abbr)
pos_prone[UNIT]

['different', 'likely', 'familiar', 'similar']

# %% [markdown]

## Strongest associations for each polarity by metric

In [ ]:
def show_metric_top(amdf: pd.DataFrame,
                    metric: str,
                    k=5, cols=[None]):
    if not any(cols):
        cols = amdf.columns
    return (pd.concat((polar_df.nlargest(k, metric)
                       for pol, polar_df in amdf.groupby('polarity')))
            .loc[:, [metric] + cols[cols != metric].to_list()]
            .reset_index(drop=True).set_index(['polarity', 'l1', 'l2'])
            )
def update_prone(exdf, pos_prone, neg_prone) -> dict: 
    prone_dict = {'positive':pos_prone, 
                  'negative': neg_prone}
    for polarity, prone in prone_dict.items():
        try: 
            polar_ex = exdf.loc[polarity, :]
        except KeyError: 
            continue
        else: 
                
            for topw in polar_ex.reset_index().l2.head(2).squeeze(): 
                if topw not in prone[UNIT]: 
                    prone[UNIT].append(topw)
            prone_dict[polarity] = prone
    return prone_dict['positive'], prone_dict['negative']

# %% [markdown]

 Top consevative log ratio $LRC$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr,"LRC")
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

LRC      quant      f  unexp_f  dP1  dP2  \
polarity l1      l2                                                      
negative never   late        6.68  not_exist  10804 6,673.71 0.64 0.10   
                 early       6.47  not_exist   4901 3,030.10 0.62 0.04   
         nobody  surprised   5.39  not_exist    259   249.72 0.59 0.04   
         nothing frustrating 5.32  not_exist   1978 1,289.02 0.64 0.02   
         none    pleased     4.83  not_exist   1298 1,130.14 0.72 0.04   

                              dP1_simple  dP2_simple        G2       N  \
polarity l1      l2                                                      
negative never   late               0.99        0.10 20,593.54  289776   
                 early              0.99        0.04  9,229.81  289776   
         nobody  surprised          0.61        0.04  1,429.36  289776   
         nothing frustrating        0.98        0.02  3,902.59  289776   
         none    pleased            0.83        0.04  4,468.11  289776   

                                  f1     f2    exp_f  
polarity l1      l2                                   
negative never   late         109723  10908 4,130.29  
                 early        109723   4941 1,870.90  
         nobody  surprised      6311    426     9.28  
         nothing frustrating   98788   2021   688.98  
         none    pleased       31021   1568   167.86

# %% [markdown]

 Top $\Delta P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1      quant     f  unexp_f  LRC  dP2  dP1_simple  \
polarity l1   l2                                                             
negative no   dunk    0.94  not_exist    33    31.96 2.45 0.00        0.97   
         none blind   0.75  not_exist   190   166.13 4.18 0.01        0.85   
              pleased 0.72  not_exist  1298 1,130.14 4.83 0.04        0.83   
         no   pink    0.72  not_exist    45    43.16 4.34 0.01        0.75   
         none deaf    0.69  not_exist    24    20.79 1.65 0.00        0.80   

                       dP2_simple       G2       N     f1    f2  exp_f  
polarity l1   l2                                                        
negative no   dunk           0.00   221.11  289776   8890    34   1.04  
         none blind          0.01   670.60  289776  31021   223  23.87  
              pleased        0.04 4,468.11  289776  31021  1568 167.86  
         no   pink           0.01   247.25  289776   8890    60   1.84  
         none deaf           0.00    78.60  289776  31021    30   3.21

# %% [markdown]

 Top conditional probability $P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1_simple      quant   f  unexp_f  LRC  dP1  \
polarity l1      l2                                                           
negative nothing amiss               1.00  not_exist  38    25.05 0.00 0.66   
                 untoward            1.00  not_exist  36    23.73 0.00 0.66   
                 demoralising        1.00  not_exist  22    14.50 0.00 0.66   
                 energizing          1.00  not_exist  21    13.84 0.00 0.66   
         never   rid                 1.00  not_exist  31    19.26 0.00 0.62   

                               dP2  dP2_simple    G2       N      f1  f2  \
polarity l1      l2                                                        
negative nothing amiss        0.00        0.00 81.80  289776   98788  38   
                 untoward     0.00        0.00 77.49  289776   98788  36   
                 demoralising 0.00        0.00 47.35  289776   98788  22   
                 energizing   0.00        0.00 45.20  289776   98788  21   
         never   rid          0.00        0.00 60.22  289776  109723  31   

                               exp_f  
polarity l1      l2                   
negative nothing amiss         12.95  
                 untoward      12.27  
                 demoralising   7.50  
                 energizing     7.16  
         never   rid           11.74

# %% [markdown]

 Top $\Delta P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2             quant      f  unexp_f  LRC  dP1  \
polarity l1       l2                                                           
negative nobody   sure      0.18         not_exist   1232 1,106.71 3.39 0.20   
         never    late      0.10         not_exist  10804 6,673.71 6.68 0.64   
         hardly   different 0.09  hedged_not_exist    519   490.87 4.40 0.33   
         scarcely different 0.08  hedged_not_exist     68    63.96 3.25 0.04   
         rarely   simple    0.05  hedged_not_exist    267   224.23 2.30 0.08   

                             dP1_simple  dP2_simple        G2       N      f1  \
polarity l1       l2                                                            
negative nobody   sure             0.21        0.20  3,867.53  289776    6311   
         never    late             0.99        0.10 20,593.54  289776  109723   
         hardly   different        0.35        0.09  2,277.50  289776    5468   
         scarcely different        0.05        0.09    264.27  289776     785   
         rarely   simple           0.10        0.06    560.14  289776    4439   

                                f2    exp_f  
polarity l1       l2                         
negative nobody   sure        5753   125.29  
         never    late       10908 4,130.29  
         hardly   different   1491    28.13  
         scarcely different   1491     4.04  
         rarely   simple      2792    42.77

# %% [markdown]

 Top conditional probability $P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2_simple             quant      f  unexp_f  \
polarity l1       l2                                                        
negative nobody   sure             0.20         not_exist   1232 1,106.71   
         never    late             0.10         not_exist  10804 6,673.71   
         hardly   different        0.09  hedged_not_exist    519   490.87   
         scarcely different        0.09  hedged_not_exist     68    63.96   
         nothing  important        0.08         not_exist   8025 3,027.92   

                             LRC  dP1  dP2  dP1_simple        G2       N  \
polarity l1       l2                                                       
negative nobody   sure      3.39 0.20 0.18        0.21  3,867.53  289776   
         never    late      6.68 0.64 0.10        0.99 20,593.54  289776   
         hardly   different 4.40 0.33 0.09        0.35  2,277.50  289776   
         scarcely different 3.25 0.04 0.08        0.05    264.27  289776   
         nothing  important 1.11 0.22 0.05        0.55  2,763.68  289776   

                                 f1     f2    exp_f  
polarity l1       l2                                 
negative nobody   sure         6311   5753   125.29  
         never    late       109723  10908 4,130.29  
         hardly   different    5468   1491    28.13  
         scarcely different     785   1491     4.04  
         nothing  important   98788  14658 4,997.08

# %% [markdown]

 Top log-likelihood $G^2$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'G2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

G2      quant      f  unexp_f  LRC  dP1  \
polarity l1      l2                                                           
negative never   late        20,593.54  not_exist  10804 6,673.71 6.68 0.64   
                 early        9,229.81  not_exist   4901 3,030.10 6.47 0.62   
         nothing wrong        7,817.55  not_exist   4856 3,040.99 3.97 0.58   
         none    pleased      4,468.11  not_exist   1298 1,130.14 4.83 0.72   
         nothing frustrating  3,902.59  not_exist   1978 1,289.02 5.32 0.64   

                              dP2  dP1_simple  dP2_simple       N      f1  \
polarity l1      l2                                                         
negative never   late        0.10        0.99        0.10  289776  109723   
                 early       0.04        0.99        0.04  289776  109723   
         nothing wrong       0.05        0.91        0.05  289776   98788   
         none    pleased     0.04        0.83        0.04  289776   31021   
         nothing frustrating 0.02        0.98        0.02  289776   98788   

                                 f2    exp_f  
polarity l1      l2                           
negative never   late         10908 4,130.29  
                 early         4941 1,870.90  
         nothing wrong         5324 1,815.01  
         none    pleased       1568   167.86  
         nothing frustrating   2021   688.98

In [ ]:
# %%

sig_trig_abbr = ex_trig_abbr.loc[ex_trig_abbr.LRC.abs() > 1, :]
def show_prone_ex(amdf, prone_list, list_index:int=0):
    try: 
        example = prone_list[list_index]

    except IndexError: 
        return list_index + 1, f'No {list_index}th entry'
    else:
        print(f'>> {example} <<')
        return list_index+ 1, amdf.filter(regex=f'~{example}$', axis=0).iloc[:10, :]

# %% [markdown]

 ---
 Significant Examples for Items demonstrated positive polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone = pos_prone[UNIT]
prone

['different', 'likely', 'familiar', 'similar']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> different <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
hardly~different,negative,hedged_not_exist,519,490.87,4.40,0.33,0.09,0.35,0.09,"2,277.50",289776,5468,1491,28.13,hardly,different
scarcely~different,negative,hedged_not_exist,68,63.96,3.25,0.04,0.08,0.05,0.09,264.27,289776,785,1491,4.04,scarcely,different
barely~different,negative,hedged_not_exist,39,32.65,1.46,0.02,0.03,0.03,0.03,77.83,289776,1235,1491,6.35,barely,different
never~different,negative,not_exist,78,-486.56,-2.60,-0.33,-0.01,0.05,0.00,-887.64,289776,109723,1491,564.56,never,different


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> likely <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nor~likely,negative,not_exist,148,105.78,1.35,0.13,0.01,0.19,0.01,176.27,289776,15546,787,42.22,nor,likely
never~likely,negative,not_exist,100,-198.00,-1.27,-0.25,-0.00,0.13,0.00,-249.30,289776,109723,787,298.00,never,likely


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> familiar <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> similar <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


# %% [markdown]

 ---
## Examples for Items demonstrating *Negative* polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone_list = neg_prone[UNIT]
prone = sig_trig_abbr.copy().loc[sig_trig_abbr.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
prone

['important',
 'late',
 'sure',
 'early',
 'frustrating',
 'evident',
 'fancy',
 'alone',
 'blind',
 'aggravating',
 'shabby',
 'dunk']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> important <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nothing~important,negative,not_exist,8025,"3,027.92",1.11,0.22,0.05,0.55,0.08,"2,763.68",289776,98788,14658,"4,997.08",nothing,important
hardly~important,negative,hedged_not_exist,72,-204.59,-1.09,-0.01,-0.04,0.00,0.01,-226.39,289776,5468,14658,276.59,hardly,important
rarely~important,negative,hedged_not_exist,51,-173.54,-1.12,-0.01,-0.04,0.00,0.01,-205.14,289776,4439,14658,224.54,rarely,important
nobody~important,negative,not_exist,71,-248.23,-1.31,-0.02,-0.04,0.00,0.01,-297.66,289776,6311,14658,319.23,nobody,important
neither~important,negative,not_exist,73,-253.67,-1.32,-0.02,-0.04,0.00,0.01,-303.62,289776,6458,14658,326.67,neither,important
nor~important,negative,not_exist,140,-646.38,-1.92,-0.05,-0.04,0.01,0.01,-868.70,289776,15546,14658,786.38,nor,important


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> late <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~late,negative,not_exist,10804,"6,673.71",6.68,0.64,0.10,0.99,0.10,"20,593.54",289776,109723,10908,"4,130.29",never,late
rarely~late,negative,hedged_not_exist,20,-147.10,-1.42,-0.01,-0.03,0.00,0.00,-216.37,289776,4439,10908,167.10,rarely,late
nor~late,negative,not_exist,27,-558.20,-3.08,-0.05,-0.04,0.00,0.00,"-1,001.69",289776,15546,10908,585.20,nor,late
nothing~late,negative,not_exist,22,"-3,696.66",-6.41,-0.35,-0.06,0.00,0.00,"-9,023.78",289776,98788,10908,"3,718.66",nothing,late


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> sure <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nobody~sure,negative,not_exist,1232,"1,106.71",3.39,0.20,0.18,0.21,0.20,"3,867.53",289776,6311,5753,125.29,nobody,sure
never~sure,negative,not_exist,4069,"1,890.64",1.77,0.34,0.03,0.71,0.04,"2,603.37",289776,109723,5753,"2,178.36",never,sure
none~sure,negative,not_exist,175,-440.87,-1.36,-0.08,-0.02,0.03,0.01,-485.28,289776,31021,5753,615.87,none,sure
nor~sure,negative,not_exist,58,-250.64,-1.49,-0.04,-0.02,0.01,0.00,-323.06,289776,15546,5753,308.64,nor,sure
nothing~sure,negative,not_exist,53,"-1,908.26",-4.77,-0.34,-0.03,0.01,0.00,"-4,321.23",289776,98788,5753,"1,961.26",nothing,sure


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> early <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~early,negative,not_exist,4901,"3,030.10",6.47,0.62,0.04,0.99,0.04,"9,229.81",289776,109723,4941,"1,870.90",never,early


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> frustrating <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nothing~frustrating,negative,not_exist,1978,"1,289.02",5.32,0.64,0.02,0.98,0.02,"3,902.59",289776,98788,2021,688.98,nothing,frustrating


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> evident <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~evident,negative,not_exist,1221,670.07,2.57,0.46,0.01,0.84,0.01,"1,317.46",289776,109723,1455,550.93,never,evident
nothing~evident,negative,not_exist,60,-436.03,-2.60,-0.30,-0.01,0.04,0.00,-795.11,289776,98788,1455,496.03,nothing,evident


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> fancy <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nothing~fancy,negative,not_exist,746,479.75,4.07,0.62,0.01,0.96,0.01,"1,352.55",289776,98788,781,266.25,nothing,fancy


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> alone <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~alone,negative,not_exist,508,294.06,2.83,0.52,0.00,0.90,0.00,672.67,289776,109723,565,213.94,never,alone


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> blind <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
none~blind,negative,not_exist,190,166.13,4.18,0.75,0.01,0.85,0.01,670.60,289776,31021,223,23.87,none,blind


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> aggravating <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nothing~aggravating,negative,not_exist,104,66.16,1.93,0.60,0.00,0.94,0.00,177.50,289776,98788,111,37.84,nothing,aggravating


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> shabby <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
none~shabby,negative,not_exist,68,57.94,2.73,0.62,0.00,0.72,0.00,199.03,289776,31021,94,10.06,none,shabby


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, [r'exactly\w*'] if UNIT!='Adj' else [r'un\w+'])
exdf

>> un\w+ <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nothing~unusual,negative,not_exist,433,247.54,2.12,0.46,0.00,0.80,0.00,474.94,289776,98788,544,185.46,nothing,unusual
nothing~unique,negative,not_exist,296,142.25,1.15,0.32,0.00,0.66,0.00,186.23,289776,98788,451,153.75,nothing,unique
